In [9]:
import re
import random
from tqdm.notebook import tqdm
from tika import parser

import warnings
from pathlib import Path

import spacy
from thinc.api import SGD
from spacy.tokens import DocBin

In [10]:
# pdf_file = '/Users/gopinath.balu/Downloads/text_extraction/data/Use caes NLP.pdf'
pdf_file = '/home/admin/gopi/Workspace/text_extraction/data/Use caes NLP.pdf'

In [3]:
raw = parser.from_file(pdf_file)
content = raw['content']

In [4]:
remove_newline = lambda x: x.replace('\n', ' ').strip()

In [5]:
actual_content = remove_newline(content)

In [11]:
edited_content = "Numéro de TVA : BE 0425.888.396 RPR GENT KBC IBAN BE50 4459 6389 4118 BIC KREDBEBB  ING IBAN BE98 3900 3232 4293 BIC BBRUBEBB  BNP IBAN BE78 2900 1606 0086 BIC GEBABEBB  BEL IBAN BE30 5645 1378 2011 BIC GKCCBEBB 1 / 2  pgb-Europe nv  Gontrode Heirweg 170  9090 Melle - Belgium  Tél. +32 (0)9 272 70 70  info@pgb-europe.com  Fax +32 (0)9 272 70 99  www.pgb-europe.com  Société Certifiée ISO 9001  CONFIRMATION D'ORDRE - 301437  Date du document 02/05/2022 Numéro de client 1000293 Numéro de TVA client BE0415809702 Votre réf. 13639 E-mail florine.fernez@plasticentre.b  e Tél. 069532060  Plasticentre sa  Lid Menouquin :115  /PLH   Chée de Russeignies 11  9600 Renaix  Adresse de facturation Menouquin sv Pro  Groep Meno0  Vieux Chemin de Thines 14  1400 Nivelles  Adresse de livraison Plasticentre  003147999  Rue De L'Artisanat 11  7900 Leuze-en-Hainaut  Vous pourrez trouver nos conditions générales de vente et de livraison sur ce  document ou via le lien suivant :   https://www.pgb-europe.com/fr-fr/legal/terms-and-conditions  Conditions de paiement Dû en 60 jours Conditions de livraison CPT (Port payé) Leuze-en-Hainaut Valeur d'envoi franco 250€     POS. DESCRIPTION QUANTITE   COMMANDEE QUANTITE   CONFIRMEE PRIX /  UNITE  NET /  UNITE  MONTANT  NET  10  PFWVTG001005000803 / PWVTGG50080 Z (*) Vis à panneaux PFS+ TF-T * 5,00x80 Zn (T25)  double tête plate six lobes int. avec nervures  filet partiel - pointe coupante T17  emballage standard QTE: 200 Pce  Demandé: 2.000 Pce Arrondi:  2.400 Pce  Stock: 2.400 Pce  Pce  -2 %  Pce  117,53 €  20  PFWVTV001004500603 / PWVTVK45060 Z (*) Vis à panneaux PFS+ TF-T * 4,50x60 Zn (T20)  double tête plate six lobes int. avec nervures  filet total - pointe coupante T17  emballage petit QTE: 200 Pce  2.400 Pce Stock: 2.400 Pce   Pce   Pce  68,71 €  Sous Total 188,64 €  Remise cumulée -2,40 €  Montant net 186,24 €  TVA (21%) 39,11 €  Montant total 225,35 €  Poids brut 27,600 KG CONFIRMATION D'ORDRE - 301437  Date de document 02/05/2022  2 / 2  Nous restons à votre disposition pour tout renseignement complémentaire.  Sofie Van Dorpe Sales sofie.van.dorpe@pgb-europe.com +32 92727072  (*) Sans Chrome VI conformément à la directive RoHS (directive 2011/65 /UE)."

In [7]:
#items_to_extract = ['Art. client: 924734', '49,97 €  / 1.000', '48,97 €  / 1.000', 'Art. client: 929735', '28,63 €  / 1.000', '28,63 €  / 1.000']

In [12]:
art_clients = [f'{random.randint(10000, 999999)}' for i in range(100)]
prices1 = [f'{random.randint(1, 999)},{random.randint(0, 99)} / 1.000' for i in range(100)]
prices2 = [f'{random.randint(1, 999)},{random.randint(0, 99)} / 1.000' for i in range(100)]

In [13]:
global info_corpus
info_corpus = list(zip(art_clients, prices1, prices2))
# info_corpus = info_corpus[:10]

In [14]:
def shuffle_word(input_sentence: str) -> str:
    """Shuffle the input string

    Args:
        word (str): Input string

    Returns:
        str: Shuffled output
    """

    words = input_sentence.split(' ')
    random.shuffle(words)
    return ' '.join(words)

def get_items(n: int) -> list:
    """Get generated info from the list

    Args:
        n (int): N no of items to fetch

    Returns:
        list: return list
    """

    if len(info_corpus) > n:
        result = random.sample(info_corpus, n)
        for item in result:
            info_corpus.remove(item)
    else:
        result = random.sample(info_corpus, len(info_corpus))
        for item in result:
            info_corpus.remove(item)
    return result

def insert_entries(doc_content: str, entities_list: list) -> str:
    """Insert given entities into the document content

    Args:
        doc_content (str): Document content
        entities_list (list): List of entities

    Returns:
        str: Document with the inserted entities
    """
    entity_label_list = list()
    return_entity_dict = dict()
    doc_length = len(doc_content.split())
    randidx = random.randint(50, int(doc_length/2))
    
    for artid, price1, price2 in entities_list:
        doc_word_list = doc_content.split(' ')
        doc_word_list.insert(randidx, artid)
        doc_word_list.insert(randidx+random.choice([32, 48]), price1)
        doc_word_list.insert(randidx+random.choice([64, 128]), price2)
        doc_content = ' '.join([str(i) for i in doc_word_list])
        randidx = randidx + random.choice([32, 64])
    
    for items in entities_list:
        for item in items:
            if '/' not in item.lower():
                start, end = re.search(item, doc_content).span()
                entity_label_list.append((start, end, 'Art ID'))
                pass
            else:
                start, end = re.search(item, doc_content).span()
                entity_label_list.append((start, end, 'Price per Unit'))
    return_entity_dict['entities'] = entity_label_list
        
        
    return [doc_content, return_entity_dict]

In [15]:
def generate_synthetic_data(doc_content: str, data_corpus: list) -> None:
    """To generate synthetic data to train SpaCy custom NER model. 
    This returns None but generates a training data.

    Args:
        doc_content (str): Input document content, which will be 
                           modified.
        data_corpus (list): Input list of items to be placed 
                            randomly in the document.
    """
    


    data_list = list()
    len_doc = len(doc_content)

    while len(info_corpus) > 0:
        no_of_samples = 1 #random.randint(1, 2)
        # new_doc_content = shuffle_word(doc_content)
        new_doc_content = doc_content
        entities = get_items(no_of_samples)
        data_list.append(insert_entries(new_doc_content, entities))
        
    return data_list

TRAINING_DATA = generate_synthetic_data(edited_content, info_corpus)


In [16]:
def convert(lang: str, TRAINING_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAINING_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)

In [17]:
# convert("fr", TRAINING_DATA, "/Users/gopinath.balu/Downloads/text_extraction/data/fr_model/train.spacy")
convert("fr", TRAINING_DATA, "/home/admin/gopi/Workspace/text_extraction/data/fr_model/train.spacy")

In [18]:
model = 'fr_core_news_md'
optimizer = SGD(learn_rate=0.001, L2=1e-6, grad_clip=1.0)
losses = {}

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('fr')  
    print("Created blank 'fr' model")

if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
    ner.add_label("O")
else:
    ner = nlp.get_pipe('ner')

Loaded model 'fr_core_news_md'


In [20]:
from spacy.training.example import Example

for batch in spacy.util.minibatch(TRAINING_DATA, size=2):
    for text, annotations in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        # nlp.update([example], sgd=optimizer, losses=losses, drop=0.3)
        nlp.update([example], losses=losses, drop=0.3)
        print("Losses", losses)

# nlp.to_disk('/Users/gopinath.balu/Downloads/text_extraction/output/fr_model')
nlp.to_disk('/home/admin/gopi/Workspace/text_extraction/output/fr_model')

Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 90.84170774878909}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 350.4345183795874}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 398.3759510827574}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 439.08778427430116}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 470.8112873588125}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 488.74916693893084}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 498.5974723671322}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 506.5360108731539}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 514.111037630536}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 520.4150380986607}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 526.6514368975318}
Losses {'tok2vec': 0.0, 'morphologizer': 0

In [22]:
doc = nlp(edited_content)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities []
